In [1]:
import pandas as pd
from google.colab import drive
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
drive.mount('/content/drive')

Mounted at /content/drive


Load the initial dataset (players21)

In [2]:
players21=pd.read_csv('/content/drive/My Drive/Colab Notebooks/players_21.csv')

Getting a Birds-eye view of the dataset

In [3]:
players21.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,103500000.0,560000.0,33,...,52+3,52+3,52+3,62+3,19+3,https://cdn.sofifa.net/players/158/023/21_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",92,92,63000000.0,220000.0,35,...,54+3,54+3,54+3,61+3,20+3,https://cdn.sofifa.net/players/020/801/21_120.png,https://cdn.sofifa.net/teams/45/60.png,https://cdn.sofifa.net/flags/it.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
2,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,91,91,111000000.0,240000.0,31,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/21_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,NaN,https://cdn.sofifa.net/flags/pl.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,132000000.0,270000.0,28,...,49+3,49+3,49+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/21_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CAM, CM",91,91,129000000.0,370000.0,29,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/21_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png


In [4]:
players21.info(verbose=True,null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 110 columns):
 #    Column                       Non-Null Count  Dtype  
---   ------                       --------------  -----  
 0    sofifa_id                    18944 non-null  int64  
 1    player_url                   18944 non-null  object 
 2    short_name                   18944 non-null  object 
 3    long_name                    18944 non-null  object 
 4    player_positions             18944 non-null  object 
 5    overall                      18944 non-null  int64  
 6    potential                    18944 non-null  int64  
 7    value_eur                    18707 non-null  float64
 8    wage_eur                     18719 non-null  float64
 9    age                          18944 non-null  int64  
 10   dob                          18944 non-null  object 
 11   height_cm                    18944 non-null  int64  
 12   weight_kg                    18944 non-null  int64  
 13  

<ipython-input-4-84bd599bb94c>:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  players21.info(verbose=True,null_counts=True)


Dropping Irrelevant Columns based on domain knowledge

In [5]:
irrelevant_columns = ['player_url','player_face_url', 'club_logo_url','club_flag_url','nation_logo_url','nation_flag_url','nationality_id','nationality_name','nation_team_id','club_loaned_from','club_contract_valid_until','club_jersey_number']
players21.drop(columns=irrelevant_columns, inplace=True)

Remove columns with more than 30% missing values


In [6]:
players21=players21.loc[:,players21.isnull().mean() <.3]

Separating data into numeric and non-numeric set.

In [7]:
numeric_columns = players21.select_dtypes(include=['number'])  # This selects numeric columns
categorical_columns = players21.select_dtypes(exclude=['number'])  # This selects non-numeric (categorical) columns

In [8]:
numeric_columns

,sofifa_id,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,...,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,158023,93,93,103500000.0,560000.0,33,170,72,241.0,1.0,...,75,96,32,35,24,6,11,15,14,8
1,20801,92,92,63000000.0,220000.0,35,187,83,45.0,1.0,...,84,95,28,32,24,7,11,15,14,11
2,188545,91,91,111000000.0,240000.0,31,184,80,21.0,1.0,...,88,88,35,42,19,15,6,12,8,10
3,190871,91,91,132000000.0,270000.0,28,175,68,73.0,1.0,...,92,93,35,30,29,9,9,15,15,11
4,192985,91,91,129000000.0,370000.0,29,181,70,10.0,1.0,...,84,91,68,65,53,15,13,5,10,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,257710,47,52,70000.0,1000.0,21,177,70,112165.0,1.0,...,35,40,45,56,47,12,13,8,14,6
18940,257933,47,53,70000.0,1000.0,21,174,68,112540.0,1.0,...,35,35,43,42,53,8,8,13,14,10
18941,257936,47,47,45000.0,2000.0,28,185,79,111774.0,1.0,...,36,35,38,43,45,8,5,11,5,7
18942,258736,47,67,130000.0,500.0,17,171,58,1920.0,4.0,...,50,45,18,11,13,11,13,9,9,6


In [9]:
categorical_columns

,short_name,long_name,player_positions,dob,club_name,league_name,club_position,club_joined,preferred_foot,work_rate,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",1987-06-24,FC Barcelona,Spain Primera Division,CAM,2004-07-01,Left,Medium/Low,...,65+3,65+3,65+3,66+3,62+3,52+3,52+3,52+3,62+3,19+3
1,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",1985-02-05,Juventus,Italian Serie A,LS,2018-07-10,Right,High/Low,...,61+3,61+3,61+3,65+3,61+3,54+3,54+3,54+3,61+3,20+3
2,R. Lewandowski,Robert Lewandowski,ST,1988-08-21,FC Bayern München,German 1. Bundesliga,ST,2014-07-01,Right,High/Medium,...,65+3,65+3,65+3,64+3,61+3,60+3,60+3,60+3,61+3,19+3
3,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",1992-02-05,Paris Saint-Germain,French Ligue 1,LW,2017-08-03,Right,High/Medium,...,62+3,62+3,62+3,67+3,62+3,49+3,49+3,49+3,62+3,20+3
4,K. De Bruyne,Kevin De Bruyne,"CAM, CM",1991-06-28,Manchester City,English Premier League,RCM,2015-08-30,Right,High/High,...,80+3,80+3,80+3,79+3,75+3,69+3,69+3,69+3,75+3,21+3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,Zhang Mengxuan,张梦炫,CB,1999-04-26,Chongqing Liangjiang Athletic,Chinese Super League,SUB,2020-08-01,Right,Low/Low,...,41+2,41+2,41+2,42+2,45+2,47+2,47+2,47+2,45+2,15+2
18940,Huang Wenzhou,黄文卓,CM,1999-01-07,Shanghai Port FC,Chinese Super League,RES,2020-08-01,Right,Low/Low,...,48+2,48+2,48+2,47+2,47+2,46+2,46+2,46+2,47+2,15+2
18941,Song Yue,宋岳,CM,1991-11-20,Tianjin Jinmen Tiger FC,Chinese Super League,RES,2020-08-01,Right,Low/Low,...,47,47,47,47,47,46+1,46+1,46+1,47,11+2
18942,V. Da Silva,Ivanilson Loforte Tique Da Silva,ST,2003-03-30,Oldham Athletic,English League Two,SUB,2020-08-01,Right,Medium/Medium,...,32+2,32+2,32+2,35+2,33+2,26+2,26+2,26+2,33+2,14+2


Handling missing values

In [10]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
numeric_columns = pd.DataFrame(imputer.fit_transform(numeric_columns), columns=numeric_columns.columns)
numeric_columns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 54 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   sofifa_id                    18944 non-null  float64
 1   overall                      18944 non-null  float64
 2   potential                    18944 non-null  float64
 3   value_eur                    18944 non-null  float64
 4   wage_eur                     18944 non-null  float64
 5   age                          18944 non-null  float64
 6   height_cm                    18944 non-null  float64
 7   weight_kg                    18944 non-null  float64
 8   club_team_id                 18944 non-null  float64
 9   league_level                 18944 non-null  float64
 10  weak_foot                    18944 non-null  float64
 11  skill_moves                  18944 non-null  float64
 12  international_reputation     18944 non-null  float64
 13  release_clause_e

In [11]:
categorical_columns.fillna(method='ffill', inplace=True)

In [12]:
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder
label_encoder = LabelEncoder()

# Encode each categorical column
for column in categorical_columns:
    categorical_columns[column] = label_encoder.fit_transform(categorical_columns[column])

# Now you can access the encoded DataFrame

encoded=categorical_columns
# Display the first few rows of the encoded DataFrame
encoded.head(30)


,short_name,long_name,player_positions,dob,club_name,league_name,club_position,club_joined,preferred_foot,work_rate,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,10059,10302,539,942,238,41,0,8,0,7,...,136,136,136,133,110,59,59,59,110,18
1,3261,3332,587,391,361,24,13,1135,1,1,...,110,110,110,126,104,66,66,66,104,20
2,14329,14687,560,1271,240,19,27,289,1,2,...,136,136,136,119,104,96,96,96,104,18
3,13007,12964,331,2438,461,17,14,888,1,2,...,116,116,116,139,110,52,52,52,110,20
4,8987,9626,10,2229,407,15,19,487,1,0,...,232,232,232,204,185,158,158,158,185,22
5,8080,7595,213,2761,68,41,5,300,1,8,...,30,30,30,25,25,23,23,23,25,109
6,12246,11104,213,2522,238,41,5,289,1,8,...,40,40,40,27,23,23,23,23,23,108
7,16959,17037,55,2239,396,15,8,979,1,8,...,253,253,253,208,209,258,258,258,209,20
8,15429,15188,330,2503,396,15,14,622,1,2,...,136,136,136,151,137,89,89,89,137,20
9,11916,12553,501,2567,396,15,25,857,0,2,...,157,157,157,163,143,84,84,84,143,24


Combine numeric and encoded categorical columns


In [13]:
# Concatenate them row-wise
combined_df = pd.concat([numeric_columns, encoded],axis=1)

# Reset the index if needed
players21 = combined_df.reset_index(drop=True)


In [14]:
players21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 93 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   sofifa_id                    18944 non-null  float64
 1   overall                      18944 non-null  float64
 2   potential                    18944 non-null  float64
 3   value_eur                    18944 non-null  float64
 4   wage_eur                     18944 non-null  float64
 5   age                          18944 non-null  float64
 6   height_cm                    18944 non-null  float64
 7   weight_kg                    18944 non-null  float64
 8   club_team_id                 18944 non-null  float64
 9   league_level                 18944 non-null  float64
 10  weak_foot                    18944 non-null  float64
 11  skill_moves                  18944 non-null  float64
 12  international_reputation     18944 non-null  float64
 13  release_clause_e

Preparing data for training using random forest regressor model and using future importance


In [15]:
# Prepare the data for training
X = combined_df.drop('overall', axis=1)
y = combined_df['overall']

# Create a Random Forest Regressor model
random_forest = RandomForestRegressor()
random_forest.fit(X, y)

# Get feature importances
feature_importances = random_forest.feature_importances_

# Create a DataFrame to associate feature names with their importance scores
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

# Sort the features by importance in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Display the feature importances
print(feature_importance_df)

               Feature  Importance
2            value_eur    0.671708
12  release_clause_eur    0.142544
56                 dob    0.111504
1            potential    0.049507
32  movement_reactions    0.015722
..                 ...         ...
9            weak_foot    0.000020
62           work_rate    0.000018
10         skill_moves    0.000009
61      preferred_foot    0.000006
64           real_face    0.000003

[92 rows x 2 columns]


Selecting the 10 top features and scale them


In [16]:
top_n = 10
top_features = feature_importance_df.head(top_n)['Feature'].tolist()




In [17]:
X = X[top_features]

scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)



In [18]:
# Display the scaled top features
X_scaled

,value_eur,release_clause_eur,dob,potential,movement_reactions,age,gk,sofifa_id,wage_eur,lb
0,13.073165,13.695696,-1.582441,3.586563,3.554438,1.655055,0.084727,-2.510848,27.857178,0.575984
1,7.809992,7.312715,-1.933771,3.422893,3.664174,2.080838,0.206313,-7.562115,10.662997,0.449876
2,14.047827,13.042079,-1.372662,3.259222,3.444701,1.229273,0.084727,-1.387305,11.674420,0.449876
3,16.776879,16.565484,-0.628555,3.259222,3.225227,0.590598,0.206313,-1.301683,13.191553,0.575984
4,16.387015,16.003782,-0.761819,3.259222,3.225227,0.803490,0.327900,-1.223865,18.248665,2.152338
...,...,...,...,...,...,...,...,...,...,...
18939,-0.368070,-0.432955,0.993560,-3.123941,-1.493455,-0.899642,-0.462413,1.158722,-0.412078,-0.811208
18940,-0.368070,-0.431423,0.925334,-2.960270,-1.273981,-0.899642,-0.462413,1.166931,-0.412078,-0.706118
18941,-0.371319,-0.433977,-0.674464,-3.942295,-1.932402,0.590598,-0.948759,1.167041,-0.361507,-0.727136
18942,-0.360272,-0.421926,1.754245,-0.668878,-0.944771,-1.751207,-0.583999,1.196490,-0.437363,-1.189533


In [19]:
y

0        93.0
1        92.0
2        91.0
3        91.0
4        91.0
         ... 
18939    47.0
18940    47.0
18941    47.0
18942    47.0
18943    47.0
Name: overall, Length: 18944, dtype: float64

Spliting the data to train and test

In [20]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [21]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor
import xgboost as xgb


In [22]:
param_grid_gb = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.05],
    'max_depth': [2, 3, 4]
}

param_grid_ab = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.05, 0.1]
}

param_grid_rf = {
    'n_estimators': [50, 100, 150],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

param_grid_xgb = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5]
}


In [23]:
gradient_boosting = GradientBoostingRegressor()
ada_boost = AdaBoostRegressor()
random_forest = RandomForestRegressor()
xg_boost = xgb.XGBRegressor()


Define cross-validation settings

In [24]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)


Perform hyperparameter search with cross-validation for each regressor and fitting the models to train data


In [25]:
grid_search_gb = GridSearchCV(gradient_boosting, param_grid=param_grid_gb, cv=cv, n_jobs=-1, scoring='neg_mean_absolute_error')
grid_search_ab = GridSearchCV(ada_boost, param_grid=param_grid_ab, cv=cv, n_jobs=-1, scoring='neg_mean_absolute_error')
grid_search_rf = GridSearchCV(random_forest, param_grid=param_grid_rf, cv=cv, n_jobs=-1, scoring='neg_mean_absolute_error')
grid_search_xgb = GridSearchCV(xg_boost, param_grid=param_grid_xgb, cv=cv, n_jobs=-1, scoring='neg_mean_absolute_error')


In [26]:
grid_search_gb.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05], 'max_depth': [2, 3, 4],
                         'n_estimators': [50, 100, 150]},
             scoring='neg_mean_absolute_error')

In [27]:
grid_search_ab.fit(X_train, y_train)


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=AdaBoostRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1],
                         'n_estimators': [50, 100, 150]},
             scoring='neg_mean_absolute_error')

In [28]:
grid_search_rf.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [10, 20, 30],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 150]},
             scoring='neg_mean_absolute_error')

In [29]:
grid_search_xgb.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    inte...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [3, 4, 5],
                         'n_estimators': [50, 100, 150]},
             scoring='neg_mean_absolute_error')

Getting the  best models from grid search

In [30]:
best_gb_model = grid_search_gb.best_estimator_



In [31]:
best_ab_model = grid_search_ab.best_estimator_


In [32]:
best_rf_model = grid_search_rf.best_estimator_


In [33]:
best_xgb_model = grid_search_xgb.best_estimator_

Making predictions on the testing set


In [34]:
predictions_gb = best_gb_model.predict(X_test)



In [35]:
predictions_ab = best_ab_model.predict(X_test)


In [36]:
predictions_rf = best_rf_model.predict(X_test)


In [37]:
predictions_xgb = best_xgb_model.predict(X_test)

Calculate evaluation metrics


In [38]:
mae_gb = mean_absolute_error(y_test, predictions_gb)
mae_ab = mean_absolute_error(y_test, predictions_ab)
mae_rf = mean_absolute_error(y_test, predictions_rf)
mae_xgb = mean_absolute_error(y_test, predictions_xgb)
rmse_gb = mean_squared_error(y_test, predictions_gb, squared=False)
rmse_ab = mean_squared_error(y_test, predictions_ab, squared=False)
rmse_rf = mean_squared_error(y_test, predictions_rf, squared=False)
rmse_xgb = mean_squared_error(y_test, predictions_xgb, squared=False)

print(f"Mean Absolute Error (Gradient Boosting): {mae_gb:.2f}")
print(f"Mean Absolute Error (AdaBoost): {mae_ab:.2f}")
print(f"Mean Absolute Error (Random Forest): {mae_rf:.2f}")
print(f"Mean Absolute Error (XGBoost): {mae_xgb:.2f}")
print(f"Root Mean Squared Error (Gradient Boosting): {rmse_gb:.2f}")
print(f"Root Mean Squared Error (AdaBoost): {rmse_ab:.2f}")
print(f"Root Mean Squared Error (Random Forest): {rmse_rf:.2f}")
print(f"Root Mean Squared Error (XGBoost): {rmse_xgb:.2f}")


Mean Absolute Error (Gradient Boosting): 0.48
Mean Absolute Error (AdaBoost): 1.43
Mean Absolute Error (Random Forest): 0.22
Mean Absolute Error (XGBoost): 0.36
Root Mean Squared Error (Gradient Boosting): 0.68
Root Mean Squared Error (AdaBoost): 1.82
Root Mean Squared Error (Random Forest): 0.43
Root Mean Squared Error (XGBoost): 0.52


Load the new dataset (players22)


In [39]:
players22=pd.read_csv('/content/drive/My Drive/Colab Notebooks/players_22.csv')

<ipython-input-39-816307c10e1d>:1: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  players22=pd.read_csv('/content/drive/My Drive/Colab Notebooks/players_22.csv')


In [40]:
players22.head()

players22.info(verbose=True,null_counts=True)



<ipython-input-40-d2848f5399e7>:3: FutureWarning: null_counts is deprecated. Use show_counts instead
  players22.info(verbose=True,null_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 110 columns):
 #    Column                       Non-Null Count  Dtype  
---   ------                       --------------  -----  
 0    sofifa_id                    19239 non-null  int64  
 1    player_url                   19239 non-null  object 
 2    short_name                   19239 non-null  object 
 3    long_name                    19239 non-null  object 
 4    player_positions             19239 non-null  object 
 5    overall                      19239 non-null  int64  
 6    potential                    19239 non-null  int64  
 7    value_eur                    19165 non-null  float64
 8    wage_eur                     19178 non-null  float64
 9    age                          19239 non-null  int64  
 10   dob                          19239 non-null  object 
 11   height_cm                    19239 non-null  int64  
 12   weight_kg                    19239 non-null  int64  
 13  

Removing the irrelevant columns

In [41]:
irrelevant_columns = ['player_url','player_face_url', 'club_logo_url','club_flag_url','nation_logo_url','nation_flag_url','nationality_id','nationality_name','nation_team_id','club_loaned_from','club_contract_valid_until','club_jersey_number']
players22.drop(columns=irrelevant_columns, inplace=True)


In [42]:
players22=players22.loc[:,players22.isnull().mean() <.3]


Separating data, imputing and encoding them respectively

In [43]:
numeric_columns = players22.select_dtypes(include=['number'])  # This selects numeric columns
categorical_columns = players22.select_dtypes(exclude=['number'])  # This selects non-numeric (categorical) columns

numeric_columns

,sofifa_id,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,...,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,158023,93,93,78000000.0,320000.0,34,170,72,73.0,1.0,...,75,96,20,35,24,6,11,15,14,8
1,188545,92,92,119500000.0,270000.0,32,185,81,21.0,1.0,...,90,88,35,42,19,15,6,12,8,10
2,20801,91,91,45000000.0,270000.0,36,187,83,11.0,1.0,...,88,95,24,32,24,7,11,15,14,11
3,190871,91,91,129000000.0,270000.0,29,175,68,73.0,1.0,...,93,93,35,32,29,9,9,15,15,11
4,192985,91,91,125500000.0,350000.0,30,181,70,10.0,1.0,...,83,89,68,65,53,15,13,5,10,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,261962,47,52,70000.0,1000.0,22,180,64,112541.0,1.0,...,42,37,38,43,48,6,10,5,15,13
19235,262040,47,59,110000.0,500.0,19,175,70,445.0,1.0,...,44,47,37,44,47,11,12,6,8,10
19236,262760,47,55,100000.0,500.0,21,178,72,111131.0,1.0,...,39,36,38,44,48,8,6,7,10,6
19237,262820,47,60,110000.0,500.0,19,173,66,111131.0,1.0,...,49,47,10,14,11,7,10,7,14,15


In [44]:
categorical_columns

,short_name,long_name,player_positions,dob,club_name,league_name,club_position,club_joined,preferred_foot,work_rate,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",1987-06-24,Paris Saint-Germain,French Ligue 1,RW,2021-08-10,Left,Medium/Low,...,64+3,64+3,64+3,66+3,61+3,50+3,50+3,50+3,61+3,19+3
1,R. Lewandowski,Robert Lewandowski,ST,1988-08-21,FC Bayern München,German 1. Bundesliga,ST,2014-07-01,Right,High/Medium,...,66+3,66+3,66+3,64+3,61+3,60+3,60+3,60+3,61+3,19+3
2,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",1985-02-05,Manchester United,English Premier League,ST,2021-08-27,Right,High/Low,...,59+3,59+3,59+3,63+3,60+3,53+3,53+3,53+3,60+3,20+3
3,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",1992-02-05,Paris Saint-Germain,French Ligue 1,LW,2017-08-03,Right,High/Medium,...,63+3,63+3,63+3,67+3,62+3,50+3,50+3,50+3,62+3,20+3
4,K. De Bruyne,Kevin De Bruyne,"CM, CAM",1991-06-28,Manchester City,English Premier League,RCM,2015-08-30,Right,High/High,...,80+3,80+3,80+3,79+3,75+3,69+3,69+3,69+3,75+3,21+3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,Song Defu,宋德福,CDM,1999-01-02,Wuhan FC,Chinese Super League,SUB,2021-04-01,Right,Medium/Medium,...,46+2,46+2,46+2,48+2,48+2,46+2,46+2,46+2,48+2,15+2
19235,C. Porter,Caoimhin Porter,CM,2002-03-18,Derry City,Rep. Ireland Airtricity League,RES,2021-02-22,Right,Medium/Medium,...,46+2,46+2,46+2,49+2,48+2,44+2,44+2,44+2,48+2,14+2
19236,N. Logue,Nathan Logue-Cunningham,CM,2000-02-07,Finn Harps,Rep. Ireland Airtricity League,RES,2021-06-18,Right,Medium/Medium,...,46+2,46+2,46+2,47+2,47+2,45+2,45+2,45+2,47+2,12+2
19237,L. Rudden,Luke Rudden,ST,2002-04-23,Finn Harps,Rep. Ireland Airtricity League,RES,2021-06-18,Right,Medium/Medium,...,29+2,29+2,29+2,33+2,32+2,26+2,26+2,26+2,32+2,15+2


In [45]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
numeric_columns = pd.DataFrame(imputer.fit_transform(numeric_columns), columns=numeric_columns.columns)
numeric_columns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 54 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   sofifa_id                    19239 non-null  float64
 1   overall                      19239 non-null  float64
 2   potential                    19239 non-null  float64
 3   value_eur                    19239 non-null  float64
 4   wage_eur                     19239 non-null  float64
 5   age                          19239 non-null  float64
 6   height_cm                    19239 non-null  float64
 7   weight_kg                    19239 non-null  float64
 8   club_team_id                 19239 non-null  float64
 9   league_level                 19239 non-null  float64
 10  weak_foot                    19239 non-null  float64
 11  skill_moves                  19239 non-null  float64
 12  international_reputation     19239 non-null  float64
 13  release_clause_e

In [46]:
categorical_columns.fillna(method='ffill', inplace=True)

In [47]:
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder
label_encoder = LabelEncoder()

# Encode each categorical column
for column in categorical_columns:
    categorical_columns[column] = label_encoder.fit_transform(categorical_columns[column])

# Now you can access the encoded DataFrame

encoded=categorical_columns
# Display the first few rows of the encoded DataFrame
encoded.head(30)


,short_name,long_name,player_positions,dob,club_name,league_name,club_position,club_joined,preferred_foot,work_rate,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,10117,10421,604,719,470,19,25,1808,0,7,...,119,119,119,117,99,56,56,56,99,18
1,14468,14882,635,1016,241,21,27,210,1,2,...,131,131,131,107,99,105,105,105,99,18
2,3288,3379,658,266,412,17,27,1825,1,1,...,88,88,88,102,93,68,68,68,93,20
3,13097,13094,372,2143,470,19,14,652,1,2,...,113,113,113,123,104,56,56,56,104,20
4,9005,9692,168,1940,411,17,19,353,1,0,...,217,217,217,181,168,164,164,164,168,22
5,8125,7656,241,2464,69,45,5,220,1,8,...,30,30,30,24,26,25,25,25,26,112
6,9225,9992,658,4559,470,19,27,847,1,1,...,113,113,113,123,109,72,72,72,109,16
7,11712,11195,241,449,241,21,5,60,1,8,...,43,43,43,34,32,27,27,27,32,110
8,12278,11242,241,2225,239,45,5,210,1,8,...,39,39,39,26,24,25,25,25,24,111
9,6436,6625,635,2657,630,17,27,47,1,0,...,145,145,145,123,114,110,110,110,114,20


Combining the numeric alongside the categorical columns

In [50]:
# Concatenate them row-wise
combined_df22 = pd.concat([numeric_columns, encoded],axis=1)

# Reset the index if needed
players22 = combined_df22.reset_index(drop=True)

In [51]:
players22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 93 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   sofifa_id                    19239 non-null  float64
 1   overall                      19239 non-null  float64
 2   potential                    19239 non-null  float64
 3   value_eur                    19239 non-null  float64
 4   wage_eur                     19239 non-null  float64
 5   age                          19239 non-null  float64
 6   height_cm                    19239 non-null  float64
 7   weight_kg                    19239 non-null  float64
 8   club_team_id                 19239 non-null  float64
 9   league_level                 19239 non-null  float64
 10  weak_foot                    19239 non-null  float64
 11  skill_moves                  19239 non-null  float64
 12  international_reputation     19239 non-null  float64
 13  release_clause_e

Preparing the data for training, finding the top features and scaling

In [52]:
# Prepare the data for training
X22 = combined_df22.drop('overall', axis=1)
y22 = combined_df22['overall']

In [53]:
# Select the top features and scale them
X22 = X22[top_features]

scaled_22 = StandardScaler()
X22_scaled = pd.DataFrame(scaled_22.fit_transform(X22), columns=X22.columns)

Making predictions based on the datasets and calculating evaluation metrics for the new dataset


In [54]:

predictions_gb = best_gb_model.predict(X22_scaled)
predictions_ab = best_ab_model.predict(X22_scaled)
predictions_rf = best_rf_model.predict(X22_scaled)
predictions_xgb = best_xgb_model.predict(X22_scaled)


mae_gb = mean_absolute_error(y22, predictions_gb)
mae_ab = mean_absolute_error(y22, predictions_ab)
mae_rf = mean_absolute_error(y22, predictions_rf)
mae_xgb = mean_absolute_error(y22, predictions_xgb)
rmse_gb = mean_squared_error(y22, predictions_gb, squared=False)
rmse_ab = mean_squared_error(y22, predictions_ab, squared=False)
rmse_rf = mean_squared_error(y22, predictions_rf, squared=False)
rmse_xgb = mean_squared_error(y22, predictions_xgb, squared=False)

print(f"Mean Absolute Error (Gradient Boosting): {mae_gb:.2f}")
print(f"Mean Absolute Error (AdaBoost): {mae_ab:.2f}")
print(f"Mean Absolute Error (Random Forest): {mae_rf:.2f}")
print(f"Mean Absolute Error (XGBoost): {mae_xgb:.2f}")
print(f"Root Mean Squared Error (Gradient Boosting): {rmse_gb:.2f}")
print(f"Root Mean Squared Error (AdaBoost): {rmse_ab:.2f}")
print(f"Root Mean Squared Error (Random Forest): {rmse_rf:.2f}")
print(f"Root Mean Squared Error (XGBoost): {rmse_xgb:.2f}")

Mean Absolute Error (Gradient Boosting): 0.71
Mean Absolute Error (AdaBoost): 1.78
Mean Absolute Error (Random Forest): 0.55
Mean Absolute Error (XGBoost): 0.64
Root Mean Squared Error (Gradient Boosting): 0.97
Root Mean Squared Error (AdaBoost): 2.37
Root Mean Squared Error (Random Forest): 1.02
Root Mean Squared Error (XGBoost): 0.87


We ended up going with random forest Regressor with grid search and cross validation beacuse it gave a lower Mean absolute error value


Saving the best model and the scaler

In [60]:
import joblib
joblib.dump(best_rf_model,'FifaModelPred.pkl')

['FifaModelPred.pkl']

In [61]:
joblib.dump(scaled_22, 'Scaling.pkl')

['Scaling.pkl']

In [57]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   value_eur           18944 non-null  float64
 1   release_clause_eur  18944 non-null  float64
 2   dob                 18944 non-null  int64  
 3   potential           18944 non-null  float64
 4   movement_reactions  18944 non-null  float64
 5   age                 18944 non-null  float64
 6   gk                  18944 non-null  int64  
 7   sofifa_id           18944 non-null  float64
 8   wage_eur            18944 non-null  float64
 9   lb                  18944 non-null  int64  
dtypes: float64(7), int64(3)
memory usage: 1.4 MB
